In [1]:
import os
import pandas as pd
import numpy as np
import re
import time
import urllib
import json
import xlrd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
import math
import random
os.chdir("D:/数据/美国data")
from fake_useragent import UserAgent


In [164]:
df=pd.read_excel("wholesoap.xlsx",Sheet="sheet1")

In [165]:
namedf=pd.DataFrame(columns=['show','role','times'])
for i in set(list(df['show'])):
    df0=df[df['show']==i]
    names=[]
    namedict = {}
    for j in list(df0['content']):
        if str(j)!='nan':
            newname=re.findall('@!(.*?) ',j)
            newname=[x.strip(':') for x in newname]
            for key in newname:
                namedict[key] = namedict.get(key, 0) + 1
        else:
            continue
    for key in namedict.keys():
        newrow={'show':i,'role':key,'times':namedict[key]}
        namedf=namedf.append(newrow,ignore_index=True)

In [166]:
namedf

,show,role,times
0,General Hospital,Luke,22941
1,General Hospital,Lucy,59
2,General Hospital,Edward,7656
3,General Hospital,Bobbie,3890
4,General Hospital,Zander,5212
5,General Hospital,Scott,5886
6,General Hospital,Serena,13
7,General Hospital,Laura,4286
8,General Hospital,Stavros,513
9,General Hospital,Helena,2446


In [168]:
namedf1=namedf[namedf['times']>1000]
namedf1

,show,role,times
0,General Hospital,Luke,22941
2,General Hospital,Edward,7656
3,General Hospital,Bobbie,3890
4,General Hospital,Zander,5212
5,General Hospital,Scott,5886
7,General Hospital,Laura,4286
9,General Hospital,Helena,2446
10,General Hospital,Nikolas,27655
11,General Hospital,Gia,3499
12,General Hospital,Lucky,25944


In [172]:
wrong=[]
for item in range(0,len(namedf1)):
    if len(str(namedf1['url'][item]))<=3:
        i=list(namedf1['show'])[item]
        j=list(namedf1['role'])[item]
        i=i.replace("All My Childen",'All My Children')
        i=i.replace("General Hospital",'General-Hospital')
        i=i.replace("As the World Turns",'CBS As the world turns')
        i=i.replace("Bold and Beautiful",'the Bold and the Beautiful')
        i=i.replace("Young and Restless",'the Young and the Restless')
        i=i.replace("Port Charles" ,'Port-Charles')
        url='https://'+i.lower().replace(" ","")+'.fandom.com/wiki/'+str(j)
        options = webdriver.ChromeOptions()
        options.add_argument('headless')
        ua = UserAgent()
        driver=webdriver.Chrome(chrome_options=options)
        driver.get(url)
        time.sleep(np.random.randint(6))
        try:
            driver.find_element_by_css_selector("span[class='alternative-suggestion']").click()
            roleurl=driver.current_url
            namedf1['url'][item]=roleurl
            driver.quit()
        except:
            newurl=driver.current_url
            if newurl!=url:
                namedf1['url'][item]=newurl
            else:
                namedf1['url'][item]=newurl
            driver.quit()

E:\software\Anaconda\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [173]:
namedf1[namedf1['url']!=0]

,Unnamed: 0,show,role,times,url
0,0,General Hospital,Luke,22941,https://general-hospital.fandom.com/wiki/Luke_...
1,1,General Hospital,Edward,7656,https://general-hospital.fandom.com/wiki/Edward
2,2,General Hospital,Bobbie,3890,https://general-hospital.fandom.com/wiki/Bobbie
3,3,General Hospital,Zander,5212,https://general-hospital.fandom.com/wiki/Zande...
4,4,General Hospital,Scott,5886,https://general-hospital.fandom.com/wiki/Scott
5,5,General Hospital,Laura,4286,https://general-hospital.fandom.com/wiki/Laura
6,6,General Hospital,Helena,2446,https://general-hospital.fandom.com/wiki/Helena
7,7,General Hospital,Nikolas,27655,https://general-hospital.fandom.com/wiki/Nikolas
8,8,General Hospital,Gia,3499,https://general-hospital.fandom.com/wiki/Gia
9,9,General Hospital,Lucky,25944,https://general-hospital.fandom.com/wiki/Lucky


In [174]:
namedf1.to_excel("namedf1.xlsx")

In [3]:
namedf1

,show,role,times,url
0,General Hospital,Luke,22941,https://general-hospital.fandom.com/wiki/Luke_...
1,General Hospital,Edward,7656,https://general-hospital.fandom.com/wiki/Edward
2,General Hospital,Bobbie,3890,https://general-hospital.fandom.com/wiki/Bobbie
3,General Hospital,Zander,5212,https://general-hospital.fandom.com/wiki/Zande...
4,General Hospital,Scott,5886,https://general-hospital.fandom.com/wiki/Scott
5,General Hospital,Laura,4286,https://general-hospital.fandom.com/wiki/Laura
6,General Hospital,Helena,2446,https://general-hospital.fandom.com/wiki/Helena
7,General Hospital,Nikolas,27655,https://general-hospital.fandom.com/wiki/Nikolas
8,General Hospital,Gia,3499,https://general-hospital.fandom.com/wiki/Gia
9,General Hospital,Lucky,25944,https://general-hospital.fandom.com/wiki/Lucky


In [31]:
number=0
tnumber=0
for item in range(460,len(namedf1)):
    tnumber+=1
    print(tnumber)
    url=str(namedf1['url'][item])
    url=url.replace("port-charles",'general-hospital')
    if len(re.findall('/wiki/(.*)',namedf1['url'][item]))<12:
        options = webdriver.ChromeOptions()
        ua = UserAgent()
        options.add_argument('headless')
        driver=webdriver.Chrome(chrome_options=options)
        driver.get(url)
        time.sleep(np.random.randint(3))
        try:
            driver.find_element_by_css_selector("span[class='alternative-suggestion']").click()
            time.sleep(1)
            roleurl=driver.current_url
            namedf1['url'][item]=roleurl
            driver.quit()
            time.sleep(1)
            number+=1
        except:
            roleurl=driver.current_url
            namedf1['url'][item]=roleurl
            driver.quit()
            continue
    else:
        continue

1


Exception ignored in: <function Service.__del__ at 0x000001D607564D90>
Traceback (most recent call last):
  File "E:\software\Anaconda\lib\site-packages\selenium\webdriver\common\service.py", line 163, in __del__
    self.stop()
  File "E:\software\Anaconda\lib\site-packages\selenium\webdriver\common\service.py", line 135, in stop
    if self.process is None:
AttributeError: 'Service' object has no attribute 'process'
E:\software\Anaconda\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


2
3
4
5
6
7
8
9


E:\software\Anaconda\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
28

In [33]:
namedf1.to_excel("namedf1.xlsx")

In [34]:
namedf1[600:]

,show,role,times,url
600,As the World Turns,B.J.,1149,https://cbsastheworldturns.fandom.com/wiki/B.J.
601,As the World Turns,Sage,1701,https://cbsastheworldturns.fandom.com/wiki/Sag...
602,As the World Turns,Jade,5196,https://cbsastheworldturns.fandom.com/wiki/Jade
603,As the World Turns,Vienna,7482,https://cbsastheworldturns.fandom.com/wiki/Vienna
604,As the World Turns,Dallas,1124,https://cbsastheworldturns.fandom.com/wiki/Dallas
605,As the World Turns,Cleo,1232,https://cbsastheworldturns.fandom.com/wiki/Cleo
606,As the World Turns,Noah,6122,https://cbsastheworldturns.fandom.com/wiki/Noa...
607,As the World Turns,Sofie,2437,https://cbsastheworldturns.fandom.com/wiki/Sofie
608,As the World Turns,Audrey,1112,https://cbsastheworldturns.fandom.com/wiki/Aud...
609,As the World Turns,Liberty,6892,https://cbsastheworldturns.fandom.com/wiki/Lib...
